 # Runtime Measurement

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from benchmark import benchmark_csv
from benchmark.environment import start, stop, remove, IMPROVEMENTS, STATES
from os import PathLike, chdir
from pathlib import Path
from benchmark.api import (
    ApiBeforeEnvironmentIndependence,
    ApiAfterEnvironmentIndependence,
    ApiServiceMerge,
)
from time import sleep
from itertools import product
from tqdm.auto import tqdm

In [ ]:
if not "PROJECT_DIR" in globals():
    PROJECT_DIR = Path().resolve().parent
    chdir(PROJECT_DIR)

if not "DATA_DIR" in globals():
    DATA_DIR = PROJECT_DIR / "memory-usage" / "data"
    DATA_DIR.mkdir(parents=True, exist_ok=True)


def out(name: str) -> PathLike:
    return DATA_DIR / f"{name}.csv"

In [ ]:
states = list(product(IMPROVEMENTS, STATES))
states

## Clean up

Making sure nothing is running before the benchmarks.

In [ ]:
for improvement, state in tqdm(states):
    remove(improvement, state)

## Compose Benchmarks

### Cold Start

In [ ]:
for improvement, state in tqdm(states):
    benchmark_csv(
        out(f"cold-start_{improvement}_{state}"),
        func=lambda *_: start(improvement, state),
        after=lambda *_: remove(improvement, state),
    )

### Warm Start

In [ ]:
for improvement, state in tqdm(states):
    start(improvement, state)
    stop(improvement, state)
    benchmark_csv(
        out(f"warm-start_{improvement}_{state}"),
        func=lambda *_: start(improvement, state),
        after=lambda *_: remove(improvement, state),
    )
    remove(improvement, state)

### Stop

In [ ]:
for improvement, state in tqdm(states):
    benchmark_csv(
        out(f"stop_{improvement}_{state}"),
        before=lambda *_: start(improvement, state),
        func=lambda *_: stop(improvement, state),
    )
    remove(improvement, state)

### Remove

In [ ]:
for improvement, state in tqdm(states):
    benchmark_csv(
        out(f"remove_{improvement}_{state}"),
        before=lambda *_: start(improvement, state),
        func=lambda *_: remove(improvement, state),
    )

## API

In [ ]:
for improvement, state in tqdm(states):
    start(improvement, state)

    sleep(1)

    if improvement == "environment-independence":
        if state == "before":
            api = ApiBeforeEnvironmentIndependence()
        else:
            api = ApiAfterEnvironmentIndependence()
    else:
        api = ApiServiceMerge()

    api.create_user("albert", "einstein", "albert.einstein@example.com")
    api.login("albert", "einstein")

    benchmark_csv(
        out(f"create-database_{improvement}_{state}"),
        func=lambda index: api.create_database(f"Database {index+1}"),
    )

    database_id = api.create_database("Test Database")

    benchmark_csv(
        out(f"create-table_{improvement}_{state}"),
        func=lambda index: api.create_table(database_id, f"Table {index+1}"),
    )

    table_id = api.create_table(database_id, "Test Table")

    benchmark_csv(
        out(f"insert-row_{improvement}_{state}"),
        func=lambda index: api.insert_row(database_id, table_id, {"id": index + 1}),
    )

    benchmark_csv(
        out(f"update-row_{improvement}_{state}"),
        func=lambda index: api.update_row(
            database_id, table_id, {"id": index + 1}, {"id": index + 11}
        ),
    )

    benchmark_csv(
        out(f"delete-row_{improvement}_{state}"),
        func=lambda index: api.delete_row(database_id, table_id, {"id": index + 11}),
    )

    benchmark_csv(
        out(f"import-csv_{improvement}_{state}"),
        func=lambda _: api.import_csv(database_id, table_id, PROJECT_DIR / "test.csv"),
    )

    benchmark_csv(
        out(f"create-query_{improvement}_{state}"),
        func=lambda index: api.create_query(
            database_id, f"SELECT id FROM test_table WHERE id = {index+101}"
        ),
    )

    query_result = api.create_query(database_id, "SELECT id FROM test_table")

    benchmark_csv(
        out(f"execute-query_{improvement}_{state}"),
        func=lambda _: api.execute_query(database_id, query_result["id"]),
    )

    remove(improvement, state)